In [14]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [0]:
PROJECT_DIR = "/content/drive/My Drive/MachineLearning/Capstone/"
DATA_FILE_NAME = 'Data/Input Data Synthetic.xlsx'
CLEANSED_FILE_DIR = 'Data/processed/'

In [0]:
#import basic libraies
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import string
import nltk

import warnings
warnings.filterwarnings('ignore')

In [0]:
import keras
from tensorflow.keras import backend as K
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Dense, LSTM, Embedding, Dropout, Activation, Bidirectional, Flatten
from tensorflow.keras.models import Model, Sequential

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer

In [0]:
def load_doc(filename):
  if(filename.endswith('.xlsx')):
    data_df = pd.read_excel(filename, lines=True)
  elif(filename.endswith('.csv')):
    data_df = pd.read_csv(filename, keep_default_na = False)
  return data_df

In [0]:
def encode_target(target):
  le = LabelEncoder()
  return le.fit_transform(target), le

def decode_prediction(pred, encoder):
  return encoder.inverse_transform(pred) 

In [20]:
cleansed_data_df = load_doc(PROJECT_DIR+CLEANSED_FILE_DIR+'cleansed_data.csv')
print('shape of Data : ', cleansed_data_df.shape)

shape of Data :  (8500, 5)


In [21]:
cleansed_data_df.describe()

,short_description,description,combined_desc,caller,assignment_group
count,8500,8500,8500,8500,8500
unique,7482,6061,7812,2950,56
top,password reset,received job job_scheduler,windows password reset password reset,bpctwhsn kzqsbmtp,GRP_0
freq,38,252,29,810,3976


In [22]:
cleansed_data_df['assignment_group'].value_counts()

GRP_0         3976
GRP_8          661
GRP_24         289
GRP_12         257
GRP_9          252
GRP_2          241
GRP_19         215
GRP_3          200
GRP_6          184
GRP_13         145
GRP_10         140
GRP_5          129
GRP_14         118
GRP_25         116
GRP_33         107
GRP_4          100
GRP_29          97
GRP_18          88
GRP_16          85
GRP_17          81
GRP_31          69
GRP_7           68
GRP_34          62
GRP_26          56
GRP_40          45
GRP_COMMON      44
GRP_28          44
GRP_41          40
GRP_15          39
GRP_30          39
GRP_42          37
GRP_20          36
GRP_45          35
GRP_1           31
GRP_22          31
GRP_11          30
GRP_21          29
GRP_47          27
GRP_48          25
GRP_23          25
GRP_62          25
GRP_60          20
GRP_39          19
GRP_27          18
GRP_37          16
GRP_44          15
GRP_36          15
GRP_50          14
GRP_53          11
GRP_65          11
GRP_52           9
GRP_55           8
GRP_51      

In [23]:
cleansed_data_df['description_length'] = cleansed_data_df['description'].str.len()
desc_max_length = cleansed_data_df['description_length'].max()
maxlen_desc_idx = cleansed_data_df['description_length'].idxmax()
print(desc_max_length)
print(maxlen_desc_idx)
print('Desc with max length ({}) at index {}:\n {}'.format(desc_max_length,maxlen_desc_idx,cleansed_data_df['description'].loc[maxlen_desc_idx]))

3190
3965
Desc with max length (3190) at index 3965:
 source system android user unknown location unknown status field user no event log incident overview seeing company device generating server response sinkhole set probable infected asset traffic not blocked port port android device host likely infected return traffic android likely visit domain give false information order prevent use domain address resolution sinkhole traffic possible indicator infected computer reaching controller taken law enforcement research organization part mitigation effort traffic sinkhole activity administrator may curious particular domain browse signature repeated sinkhole however clear indication infection incident via high priority ticket per default would like us handle differently future see handling let us know either corresponding us via ticket ticket back soc calling us ticket sinkhole domain explicit notification via medium priority ticket no phone call auto resolve sinkhole domain directly porta

**Train & Test Data:**

In [0]:
X = cleansed_data_df['description']
y = cleansed_data_df.assignment_group
y, le = encode_target(y)

In [0]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.1, stratify=y, random_state = 10)

In [0]:
max_features = 10000
maxlen = cleansed_data_df['description_length'].max()
embedding_size = 200

In [0]:
unigram_tokenizer = TfidfVectorizer(ngram_range=(1,1), max_features = max_features).fit(X_train)

In [29]:
print('Vocabulary size :', len(unigram_tokenizer.get_feature_names()))

Vocabulary size : 4018


In [35]:
X_train_seq = unigram_tokenizer.fit_transform(X_train)
print('Shape of X_train : ', X_train_seq.shape)

Shape of X_train :  (7650, 4018)


In [0]:
#this X_train_seq can be used as input to the model

In [0]:
#X_train_seq_pad = pad_sequences(X_train_seq, maxlen = 3190)